## Imports

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

Using TensorFlow backend.


In [3]:
from sklearn.metrics import confusion_matrix

In [4]:
from time import clock

In [5]:
import time
random.seed(int(time.time()))

In [6]:
wavelen=3500
noise_rms=20.

### Essential Defs

In [7]:
def gen_synth_pulse(amp, T0, wf):
    length = len(wf)
    cc_slow = 2.5; cc_slow=cc_slow/(cc_slow+1); ##charge collection slow time constant

    cc_fast = 1./2.5; #charge collection fast time constant
    alpha_cr = 1250./(1250.+1.); #fall time of output
    alpha_rc1 = 1./2.75;
    alpha_rc2 = 1./2.75;
    step=zeros(2);charge=zeros(2);cur_s=zeros(2);cur_f=zeros(2);cr=zeros(2);rc1=zeros(2);rc2=zeros(2);
    for i in range(length):
        if i>=T0:
            step[i%2]=1.
        else:
            step[i%2]=0.
        cur_s[i%2]=cc_slow*(cur_s[(i+1)%2]+step[i%2]-step[(i+1)%2]);
        cur_f[i%2]=cc_fast*(cur_s[i%2]-cur_f[(i+1)%2])+cur_f[(i+1)%2];
        charge[i%2]=charge[(i+1)%2]+amp*cur_f[i%2]*(1./cc_slow-1.);
        cr[i%2]=alpha_cr*(cr[(i+1)%2]+charge[i%2]-charge[(i+1)%2]);
        rc1[i%2]=alpha_rc1*(cr[i%2]-rc1[(i+1)%2])+rc1[(i+1)%2];
        rc2[i%2]=alpha_rc2*(rc1[i%2]-rc2[(i+1)%2])+rc2[(i+1)%2];
        wf[i]=rc2[i%2];
    return;

In [8]:
def shuffle_in_unison(a, b):
    rng_state = random.get_state()
    random.shuffle(a)
    random.set_state(rng_state)
    random.shuffle(b)

### FFT defs

In [9]:
def fft_trapezoid(rise_time, flat_top , tau):
    length = wavelen
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)
    return res[:-100]

In [10]:
def fft_cusp(shape_time, tau):
    length=wavelen
    k=2*shape_time+1;
    length = length+k;  
    m2=1.; 
    m1=m2/(exp(1./tau)-1.);
    p = empty(length)
    q = empty(length)
    s = empty(length)
    input_ = zeros(length)
    p[0]=q[0]=s[0]=input_[0]=0.
    input_[1]=1.
    dk=0.
    dl=0.
    for i in range(1,length):
        dk=0.
        dl=0.
        dk = input_[i]
        if i>=k:
            dk -= input_[i-k]

        if i>=shape_time:
            dl += input_[i-shape_time]

        if i>=shape_time+1:
            dl -= input_[i-shape_time-1]

        
        p[i] = p[i-1]+dk-k*dl;
        q[i] = q[i-1]+m2*p[i];
        s[i] = s[i-1]+q[i]+m1*p[i];
    for i in range(length):
        s[i] = s[i]/(0.5*shape_time*(shape_time+1)*m1);
    res = fft.rfft(s)
    return res[:-1*shape_time];

### Batch Gen defs

In [11]:
def gen_batch(batchsize):
    # speedy function without rise time spread and without noise spectrum
    # also increased visibility of delay 
    energies = random.random(batchsize)*2500
    delays = random.random(batchsize)*200+8
    percent = random.random(batchsize)
    #T0s = random.normal(loc=1000, scale=30, size=batchsize).round().astype(int)
    T0s = random.randint(900,1101,batchsize)
        
    rand_choices = empty((batchsize,3))
    rand_choices[:,0] = energies
    rand_choices[:,1] = delays
    rand_choices[:,2] = percent
    
    X = empty((batchsize,wavelen))
    y = append( ones(int(batchsize/2)), zeros(batchsize-int(batchsize/2)))
    
    noise = random.normal(scale=noise_rms,size=(batchsize,wavelen))
    tmp = empty(wavelen)
    tmp2= empty(wavelen)
    for i in range(int(batchsize/2)):
        gen_synth_pulse(rand_choices[i][0]*rand_choices[i][2],T0s[i],tmp)
        gen_synth_pulse(rand_choices[i][0]*(1-rand_choices[i][2]),T0s[i]+rand_choices[i][1]/4,tmp2)
        X[i] = tmp+tmp2+noise[i]
    for i in range(int(batchsize/2),batchsize):
        gen_synth_pulse(rand_choices[i][0],T0s[i],tmp)
        X[i] = tmp+noise[i]
        
    shuffle_in_unison(X,y)
    return X.reshape(batchsize,wavelen,1),y

In [12]:
def trap_gen_batch(batchsize):
    # trapezoid filter version...
    # speedy function without rise time spread and without noise spectrum
    energies = random.random(batchsize)*2500
    delays = random.random(batchsize)*200+8
    percent = random.random(batchsize)
    #T0s = random.normal(loc=1000, scale=30, size=batchsize).round().astype(int)
    T0s = random.randint(900,1101,batchsize)

    rand_choices = empty((batchsize,3))
    rand_choices[:,0] = energies
    rand_choices[:,1] = delays
    rand_choices[:,2] = percent
    
    X = empty((batchsize,wavelen))
    y = append( ones(int(batchsize/2)), zeros(batchsize-int(batchsize/2)))
    
    noise = random.normal(scale=noise_rms,size=(batchsize,wavelen))
    tmp = empty(wavelen)
    tmp2= empty(wavelen)
    trap_ = fft_trapezoid(100,0,1250.)
    
    # generate an 1:1 ratio of pileup to no pileup then shuffle them
    for i in range(int(batchsize/2)):
        gen_synth_pulse(rand_choices[i][0]*rand_choices[i][2],T0s[i],tmp)
        gen_synth_pulse(rand_choices[i][0]*(1-rand_choices[i][2]),T0s[i]+rand_choices[i][1]/4,tmp2)
        X[i] = fft.irfft(trap_*fft.rfft(tmp+tmp2+noise[i]))
    for i in range(int(batchsize/2),batchsize):
        gen_synth_pulse(rand_choices[i][0],T0s[i],tmp)
        X[i] = fft.irfft(trap_*fft.rfft(tmp+noise[i]))
        
    shuffle_in_unison(X,y)
    return X.reshape(batchsize,wavelen,1),y

In [13]:
def cusp_gen_batch(batchsize):
    # cusp filter version...
    # speedy function without rise time spread and without noise spectrum
    energies = random.random(batchsize)*2500
    delays = random.random(batchsize)*200+8
    percent = random.random(batchsize)
    T0s = random.randint(900,1101,batchsize)

    #T0s = random.normal(loc=1000, scale=30, size=batchsize).round().astype(int)

    rand_choices = empty((batchsize,3))
    rand_choices[:,0] = energies
    rand_choices[:,1] = delays
    rand_choices[:,2] = percent
    
    X = empty((batchsize,wavelen))
    y = append( ones(int(batchsize/2)), zeros(batchsize-int(batchsize/2)))
    
    noise = random.normal(scale=noise_rms,size=(batchsize,wavelen))
    tmp = empty(wavelen)
    tmp2= empty(wavelen)
    cusp_ = fft_cusp(100,1250.)
    # generate an 1:1 ratio of pileup to no pileup then shuffle them
    for i in range(int(batchsize/2)):
        gen_synth_pulse(rand_choices[i][0]*rand_choices[i][2],T0s[i],tmp)
        gen_synth_pulse(rand_choices[i][0]*(1-rand_choices[i][2]),T0s[i]+rand_choices[i][1]/4,tmp2)
        X[i] = fft.irfft(cusp_*fft.rfft(tmp+tmp2+noise[i]))
    for i in range(int(batchsize/2),batchsize):
        gen_synth_pulse(rand_choices[i][0],T0s[i],tmp)
        X[i] = fft.irfft(cusp_*fft.rfft(tmp+noise[i]))
        
    shuffle_in_unison(X,y)
    return X.reshape(batchsize,wavelen,1),y

## Model building

In [14]:
def model_gen():
    model = Sequential()

    model.add(BatchNormalization())

    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(Conv1D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(Conv1D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

## def realistic data generators

In [15]:
def gen_synth_pulse_realistic(amp, T0, wf):
    length = len(wf)
    cc_slow = 2.5+0.4*random.normal(); cc_slow=cc_slow/(cc_slow+1); ##charge collection slow time constant

    cc_fast = 1./2.5; #charge collection fast time constant
    alpha_cr = 1250./(1250.+1.); #fall time of output
    alpha_rc1 = 1./2.75;
    alpha_rc2 = 1./2.75;
    step=zeros(2);charge=zeros(2);cur_s=zeros(2);cur_f=zeros(2);cr=zeros(2);rc1=zeros(2);rc2=zeros(2);
    for i in range(length):
        if i>=T0:
            step[i%2]=1.
        else:
            step[i%2]=0.
        cur_s[i%2]=cc_slow*(cur_s[(i+1)%2]+step[i%2]-step[(i+1)%2]);
        cur_f[i%2]=cc_fast*(cur_s[i%2]-cur_f[(i+1)%2])+cur_f[(i+1)%2];
        charge[i%2]=charge[(i+1)%2]+amp*cur_f[i%2]*(1./cc_slow-1.);
        cr[i%2]=alpha_cr*(cr[(i+1)%2]+charge[i%2]-charge[(i+1)%2]);
        rc1[i%2]=alpha_rc1*(cr[i%2]-rc1[(i+1)%2])+rc1[(i+1)%2];
        rc2[i%2]=alpha_rc2*(rc1[i%2]-rc2[(i+1)%2])+rc2[(i+1)%2];
        wf[i]=rc2[i%2];
    return;

In [16]:
def superimpose_noise(wf, stdDev):
    length=len(wf)
    fin = open('power_spectrum.dat', 'r')
    powerspectrum = array(str(fin.read()).split(), dtype=float64)
    fin.close()
    my_array = zeros((65533,2))
    my_array[0][0]= -114.962
    my_array[0][0]= 0.
    for i in range(1,32767):
        phase = 2.*pi*(random.randint(1, 10000000 + 1)%10000000)/10000000.
        my_array[i][0]=powerspectrum[i-1]*cos(phase);
        my_array[65532-i+1][0]=my_array[i][0]
        my_array[i][1]=powerspectrum[i-1]*sin(phase);
        my_array[65532-i+1][1]= -1*(my_array[i][1])
    my_array = (my_array[:,0]+my_array[:,1].astype(complex_))
    my_array = fft.irfft(my_array)
    for i in range(length):
        wf[i]= length*stdDev*my_array[i]*1.633953736 # adjusted factor here since fftw3 is unnormalized fft
    return

In [17]:
def gen_realistic_batch(batchsize):
    energies = random.random(batchsize)*2500
    delays = random.random(batchsize)*200+8
    percent = random.random(batchsize)
    T0s = random.randint(900,1101,batchsize)
    
    rand_choices = empty((batchsize,3))
    rand_choices[:,0] = energies
    rand_choices[:,1] = delays
    rand_choices[:,2] = percent
    
    X = empty((batchsize,wavelen))
    y = append(ones(int(batchsize*0.5)), zeros(batchsize-int(batchsize*0.5)))
    
    tmp = empty(wavelen)
    tmp2= empty(wavelen)
    noise = empty(wavelen)

    # generate an 1:1 ratio of pileup to no pileup then shuffle them
    for i in range(int(batchsize*0.5)):
        gen_synth_pulse_realistic(rand_choices[i][0]*rand_choices[i][2],T0s[i],tmp)
        gen_synth_pulse_realistic(rand_choices[i][0]*(1-rand_choices[i][2]),T0s[i]+rand_choices[i][1]/4,tmp2)
        superimpose_noise(noise,noise_rms)
        X[i] = tmp+tmp2+noise
    for i in range(int(batchsize*0.5),batchsize):
        gen_synth_pulse_realistic(rand_choices[i][0],T0s[i],tmp)
        superimpose_noise(noise,noise_rms)
        X[i] = tmp+noise
        
    shuffle_in_unison(X,y)
    return X.reshape(batchsize,wavelen,1),y

In [18]:
def fft_generic(X, fft_func, *args):
    batchsize = len(X)
    X_ = X.reshape(batchsize,wavelen).copy()
    fft_filter = fft_func(*args)
    for i in range(batchsize):
        X_[i] = fft.irfft(fft.rfft(X_[i])*fft_filter)
    return X_.reshape(batchsize,wavelen,1)

### Comparison

In [19]:
val_size=2**10
train_size=2**14
num_batch_size=32
num_epochs=10

### Generating Realistic Test Data

In [20]:
val_X, val_y = gen_realistic_batch(val_size)

In [21]:
val_X_trap = fft_generic(val_X, fft_trapezoid, 100,0,1250)

In [22]:
val_X_cusp = fft_generic(val_X, fft_cusp, 100,1250)

## Vanilla CNN

In [23]:
train_X, train_y = gen_batch(train_size)

In [24]:
model = model_gen()

In [25]:
model.fit(train_X,train_y, batch_size = num_batch_size, epochs=num_epochs);

Epoch 1/10
 2208/16384 [===>..........................] - ETA: 37s - loss: 0.7065 - acc: 0.4955

KeyboardInterrupt: 

In [ ]:
confusion_matrix(val_y,array(around(model.predict(val_X).flatten(),0),dtype=int64))

In [ ]:
model.evaluate(val_X, val_y)

## Trapezoid Filter CNN

In [ ]:
train_X,train_y = trap_gen_batch(train_size)

In [ ]:
model = model_gen()

In [ ]:
model.fit(train_X,train_y, batch_size = num_batch_size, epochs=num_epochs);

In [ ]:
confusion_matrix(val_y,array(around(model.predict(val_X_trap).flatten(),0),dtype=int64))

In [ ]:
model.evaluate(val_X_trap, val_y)

## Cusp Filter CNN

In [ ]:
train_X,train_y = cusp_gen_batch(train_size)

In [ ]:
model = model_gen()

In [ ]:
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs/cusp/')
cp_callback = keras.callbacks.ModelCheckpoint("./logs/cusp/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto')

In [ ]:
model.fit(train_X,train_y, batch_size = num_batch_size, epochs=num_epochs, validation_data=(val_X_cusp,val_y), callbacks=[cp_callback,tb_callback])

In [ ]:
confusion_matrix(val_y,array(around(model.predict(val_X_cusp).flatten(),0),dtype=int64))

In [ ]:
model.evaluate(val_X_cusp, val_y)